In [3]:
# Imports
import matplotlib.pyplot as plt
import csv
import pandas as pd
import numpy as np
from PIL import Image as im
import sklearn
from sklearn import linear_model, datasets, metrics, svm
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
import time
import tensorflow as tf
from tensorflow import keras
import nbconvert
from tensorflow.keras import layers, models
from sklearn.model_selection import KFold
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential

In [22]:
# Paramètres
n, m = (30, 30) #taille des images
p = 43 #nombre de classes (si on utilise toute la BDD les calculs sont longs)

In [23]:
# Fonction permettant de lire les données (fournie par les auteurs de la base de données)
def readTrafficSigns(rootpath):
    '''Reads traffic sign data for German Traffic Sign Recognition Benchmark.

    Arguments: path to the traffic sign data, for example './GTSRB/Training'
    Returns:   list of images, list of corresponding labels'''
    images = [] # images
    labels = [] # corresponding labels
    # loop over all 42 classes
    for c in range(p):
        prefix = rootpath + '/' + format(c, '05d') + '/' # subdirectory for class
        gtFile = open(prefix + 'GT-'+ format(c, '05d') + '.csv') # annotations file
        gtReader = csv.reader(gtFile, delimiter=';') # csv parser for annotations file
        next(gtReader) # skip header
        # loop over all images in current annotations file
        for row in gtReader:
            images.append(plt.imread(prefix + row[0])) # the 1th column is the filename
            labels.append(row[7]) # the 8th column is the label
        gtFile.close()
    return images, labels

In [24]:
# Lecture des données
data = (readTrafficSigns("C:/Users/guill/PycharmProjects/GTSRB/Final_Training/Images"))

# Création du DataFrame
df = pd.DataFrame(data, ["data", "target"])
df = df.transpose()

# Resizing des images
for index, row in df.iterrows():
    image = row['data']
    resized = im.fromarray(image).resize((n,m))
    resized = np.asarray(resized)
    df.at[index, 'data'] = np.array(resized)

    
# Découpage du jeu de données 
X_train, X_test, y_train, y_test = train_test_split(df["data"], df["target"], test_size=0.33)

# Conversion
X_train, X_test, y_train, y_test = X_train.tolist(), X_test.tolist(), (y_train.astype(int)).tolist(), (y_test.astype(int)).tolist()
indices = X_train.index
indices2 = X_test.index

# Création des labels en vecteurs (ex : [0, 1, 0, 0, 0]) et conversion en tenseur tensorflow
y_trainflow = []
for i in range(len(y_train)):
    L = [0]*p
    L[y_train[i]] = 1
    y_trainflow.append(L)
y_trainflow = np.stack(np.array(y_trainflow))
y_train = tf.constant(y_trainflow)

y_testflow = []
for i in range(len(y_test)):
    L = [0]*p
    L[y_test[i]] = 1
    y_testflow.append(L)
y_testflow = np.stack(np.array(y_testflow))
y_test = tf.constant(y_testflow)


# On aplatit les images et on les convertit en tenseur tensorflow 
X_train = np.stack(np.array(X_train))/255
X_train = tf.constant(X_train)

X_test = np.stack(np.array(X_test))/255
X_test = tf.constant(X_test)

In [25]:
# Lecture des données
data_v = readTrafficSigns("C:/Users/guill/PycharmProjects/GTSRB_Final_Training_Images/GTSRB/Final_Training/Images")

# Création du DataFrame
df_v = pd.DataFrame(data_v, ["data_v", "target_v"])
df_v = df_v.transpose()

# Resizing des images
for index, row in df_v.iterrows():
    image_v = row['data_v']
    resized_v = im.fromarray(image_v).resize((n,m))
    resized_v = np.asarray(resized_v)
    df_v.at[index, 'data_v'] = np.array(resized_v)

# Découpage du jeu de données
X_train_v, X_test_v, y_train_v, y_test_v = train_test_split(df_v["data_v"], df_v["target_v"], test_size=0.33)

# Conversion
X_train_v, X_test_v, y_train_v, y_test_v = X_train_v.tolist(), X_test_v.tolist(), (y_train_v.astype(int)).tolist(), (y_test_v.astype(int)).tolist()
indices_v = X_train_v.index
indices2_v = X_test_v.index

# Création des labels en vecteurs (ex : [0, 1, 0, 0, 0]) et conversion en tenseur tensorflow
y_trainflow_v = []
for i in range(len(y_train_v)):
    L_v = [0]*p
    L_v[y_train_v[i]] = 1
    y_trainflow_v.append(L_v)
y_trainflow_v = np.stack(np.array(y_trainflow_v))
y_train_v = tf.constant(y_trainflow_v)

y_testflow_v = []
for i in range(len(y_test_v)):
    L_v = [0]*p
    L_v[y_test_v[i]] = 1
    y_testflow_v.append(L_v)
y_testflow_v = np.stack(np.array(y_testflow_v))
y_test_v = tf.constant(y_testflow_v)

# On aplatit les images et on les convertit en tenseur tensorflow
X_train_v = np.stack(np.array(X_train_v))/255
X_train_v = tf.constant(X_train_v)

X_test_v = np.stack(np.array(X_test_v))/255
X_test_v = tf.constant(X_test_v)


In [26]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.preprocessing import label_binarize
import matplotlib.pyplot as plt


model = models.Sequential([
    layers.Input(shape=(n, m, 3)),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(p, activation='softmax')
])
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10)
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Accuracy : ", test_accuracy)


Epoch 1/10
821/821 [==============================] - 4s 5ms/step - loss: 1.2948 - accuracy: 0.6399
Epoch 2/10
821/821 [==============================] - 4s 5ms/step - loss: 0.2323 - accuracy: 0.9351
Epoch 3/10
821/821 [==============================] - 4s 5ms/step - loss: 0.1129 - accuracy: 0.9703
Epoch 4/10
821/821 [==============================] - 4s 5ms/step - loss: 0.0697 - accuracy: 0.9813
Epoch 5/10
821/821 [==============================] - 4s 5ms/step - loss: 0.0524 - accuracy: 0.9846
Epoch 6/10
821/821 [==============================] - 4s 5ms/step - loss: 0.0423 - accuracy: 0.9883
Epoch 7/10
821/821 [==============================] - 4s 5ms/step - loss: 0.0295 - accuracy: 0.9911
Epoch 8/10
821/821 [==============================] - 4s 5ms/step - loss: 0.0277 - accuracy: 0.9914
Epoch 9/10
821/821 [==============================] - 4s 5ms/step - loss: 0.0204 - accuracy: 0.9938
Epoch 10/10
405/405 [==============================] - 1s 2ms/step - loss: 0.1222 - accuracy: 0.9684